In [1]:
# Import libraries.
using Turing, StatsPlots, Random

In [50]:
# Declare our Turing model.
@model function agent(opponent_agent, my_action, opponent_action, opponent_agents_depth=0, depth=0, discrete_sampler = PG, discrete_sampler_hyper_param=10, num_of_iterations=10)
    if depth > 0 
        opp_action_chain = sample(opponent_agent(agent, opponent_action, my_action, depth-1, opponent_agents_depth), discrete_sampler(discrete_sampler_hyper_param), num_of_iterations, progress=false);
        opp_alpha_1 = mean(opp_action_chain[:"alpha[1]"])
        opp_alpha_2 = mean(opp_action_chain[:"alpha[2]"])
        opp_alpha_3 = 1 - opp_alpha_1 - opp_alpha_2
        alpha ~ Dirichlet([opp_alpha_3, opp_alpha_1 , opp_alpha_2])
    else
        # Our prior belief about the probability of RPS.
        alpha ~ Dirichlet(ones(3)/3)
    end
    my_action ~ Categorical(vec(alpha))
    return my_action
end

agent (generic function with 6 methods)

In [80]:
# Declare our Turing model.
@model function agent(opponent_agent, my_history, opponent_history, opponent_agents_depth=0, depth=0, discrete_sampler = PG, discrete_sampler_hyper_param=10, num_of_iterations=10)
    for i in 1:length(my_history)
        my_action = my_history[i]
        opponent_action = opponent_history[i]
        if depth > 0 
            opp_action_chain = sample(opponent_agent(agent, opponent_action, my_action, depth-1, opponent_agents_depth), discrete_sampler(discrete_sampler_hyper_param), num_of_iterations, progress=false);
            opp_alpha_1 = mean(opp_action_chain[:"alpha[1]"])
            opp_alpha_2 = mean(opp_action_chain[:"alpha[2]"])
            opp_alpha_3 = 1 - opp_alpha_1 - opp_alpha_2
            alpha ~ Dirichlet([opp_alpha_3, opp_alpha_1 , opp_alpha_2])
        else
            # Our prior belief about the probability of RPS.
            alpha ~ Dirichlet(ones(3)/3)
        end
        my_action ~ Categorical(vec(alpha))
        return my_action
    end
end

agent (generic function with 6 methods)

In [73]:
my_action = 1
opponent_action = 1
opponent_depth = 1
my_depth = 1

1

In [69]:
chain = sample(agent(agent, [my_action], [opponent_action], opponent_depth, my_depth), PG(10), 5 , progress = false)
chain

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


LoadError: InterruptException:

1


In [74]:
function maximum_likelihood_action(list_of_pairs)
    max_key = -1
    max_value = -1
    for element in list_of_pairs
        key = element[1]
        value = element[2]
        if value > max_value
            max_value = value
            max_key = key
        end
    end
    return (max_key, max_value)
end

maximum_likelihood_action (generic function with 1 method)

In [77]:
function move(agent, other_agent, my_history, other_agent_history)
    other_agent_history = length(other_agent_history) > 0 ? other_agent_history : [1]
    my_history = length(my_history) > 0 ? my_history : [1]
    chain = sample(agent(other_agent, my_history, other_agent_history, opponent_depth, my_depth), PG(1), 1, progress = false)
    alpha_1 = (1, mean(chain[:"alpha[1]"]))
    alpha_2 = (2, mean(chain[:"alpha[2]"]))
    alpha_3 = (3, 1 - alpha_1[2] - alpha_2[2])
    return maximum_likelihood_action([alpha_1, alpha_2, alpha_3])[1]
end

move (generic function with 3 methods)

In [64]:
move(agent, agent, [])

opponent_action=1 my_action=1

Chains MCMC chain (5×6×1 Array{Float64,3}):

Log evidence      = 0.0
Iterations        = 1:5
Thinning interval = 1
Chains            = 1
Samples per chain = 5
parameters        = alpha[1], alpha[2], alpha[3], my_action
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat 
      Symbol   Float64   Float64    Float64   Missing     Float64   Float64 

    alpha[1]    0.2014    0.2286     0.1022   missing   -404.8958    0.9984
    alpha[2]    0.4312    0.3909     0.1748   missing      1.6392    2.3151
    alpha[3]    0.3674    0.3048     0.1363   missing      2.7960    1.3826
   my_action    2.0000    1.0000     0.4472   missing     -2.0000    0.7746

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.0087    0.0279    0.1593    0.2407    0.5393
    alpha[2]    0.0095    0.0927    0.5242    0.5775    0.9228


2

In [78]:
function game()
    first_player = agent
    second_player = agent
    num_of_simulations = 10
    first_player_history = []
    second_player_history = []
    for i in 1:num_of_simulations
        m1 = move(first_player, second_player, first_player_history, second_player_history)
        push!(first_player_history, m1)
        m2 = move(second_player, first_player, second_player_history, first_player_history)
        push!(second_player_history, m2)
        println("in simulation $i first player chose $m1 second player chose $m2")
    end
    return history
end

game (generic function with 1 method)

In [81]:
game()

in simulation 1 first player chose 2 second player chose 3
in simulation 2 first player chose 1 second player chose 3
in simulation 3 first player chose 1 second player chose 1
in simulation 4 first player chose 1 second player chose 3
in simulation 5 first player chose 3 second player chose 2
in simulation 6 first player chose 2 second player chose 2
in simulation 7 first player chose 2 second player chose 2
in simulation 8 first player chose 2 second player chose 2
in simulation 9 first player chose 3 second player chose 1
in simulation 10 first player chose 1 second player chose 3

LoadError: UndefVarError: history not defined